# NSMC 데이터셋 불러오기
- [Naver Sentiment Movie Corpus](https://github.com/e9t/nsmc)
    - 한국어로 작성된 네이버 영화 리뷰 데이터셋
- 데이터 구성
    - size
        - train: 150,000
        - test: 50,000
    - columns
        - id
            - 네이버에서 붙인 리뷰 id
        - document
            - string, 리뷰 텍스트
        - label
            - int, 리뷰에 대한 감정 라벨
            - 긍정적: 1, 부정적: 0

In [1]:
from datasets import load_dataset

dataset = load_dataset('nsmc')

Using custom data configuration default
Reusing dataset nsmc (/aiffel/.cache/huggingface/datasets/nsmc/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3)


  0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
# 데이터 수 확인
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 150000
    })
    test: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 50000
    })
})

In [3]:
# 학습/테스트 데이터셋 분리
train_dataset = dataset['train']
test_dataset = dataset['test']

In [4]:
# 위의 5개 데이터 확인하기
for i in range(5):
    for col in train_dataset.column_names:
        print(col, ":", train_dataset[col][i])
    print()

id : 9976970
document : 아 더빙.. 진짜 짜증나네요 목소리
label : 0

id : 3819312
document : 흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
label : 1

id : 10265843
document : 너무재밓었다그래서보는것을추천한다
label : 0

id : 9045019
document : 교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정
label : 0

id : 6483659
document : 사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다
label : 1



In [ ]:
# 리뷰 문장의 길이 분포 그래프 그리기
import numpy as np
import matplotlib.pyplot as plt


train_lengths = np.array(list(map(len, train_dataset['document'])))

lengths, counts = np.unique(train_lengths, return_counts=True)
plt.bar(x=lengths, y=counts)
plt.show()

# 모델, 토크나이저 불러오기
- [klue/bert-base](https://huggingface.co/klue/bert-base)
    - KLUE 데이터셋을 pre-train한 BERT 모델

In [7]:
# model과 tokenizer 불러오기
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("klue/bert-base")
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized

In [8]:
# 불러온 model과 tokenizer 확인하기
print('model:', model.__class__)
print('tokenizer:', tokenizer.__class__)

model: <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'>
tokenizer: <class 'transformers.models.bert.tokenization_bert_fast.BertTokenizerFast'>


# 데이터 전처리
- tokenizer로 추가되는 columns
    - input_ids
        - List(int), 정수로 표현된 토큰 리스트
    - attention_mask
        - List(int), 어텐션을 진행할 토큰 마스크
        - 문장 토큰과 패딩 토큰을 구분

In [9]:
# 데이터를 토큰화하는 함수
def transform(data):
    return tokenizer(
        data['document'],
        truncation=True,
        padding='max_length',
        return_token_type_ids=False,
    )

In [10]:
# transform 데이터 테스트
for i in range(5):
    tokens = transform(train_dataset[i])
    print(tokens)

{'input_ids': [2, 1376, 831, 2604, 18, 18, 4229, 9801, 2075, 2203, 2182, 4243, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [11]:
# 데이터셋 토큰화
train_dataset = train_dataset.map(transform, batched=True)
test_dataset = test_dataset.map(transform, batched=True)

Loading cached processed dataset at /aiffel/.cache/huggingface/datasets/nsmc/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3/cache-5593c927741c5627.arrow
Loading cached processed dataset at /aiffel/.cache/huggingface/datasets/nsmc/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3/cache-7c8a59eb7687747e.arrow


In [12]:
# 전처리된 데이터셋 확인
train_dataset

Dataset({
    features: ['attention_mask', 'document', 'id', 'input_ids', 'label'],
    num_rows: 150000
})

# 모델 학습

## Bucketing, Dynamic Padding 없음

In [13]:
import os
import numpy as np
from transformers import Trainer, TrainingArguments

output_dir = os.getenv('HOME') + '/aiffel/transformers'

training_arguments = TrainingArguments(
    output_dir,
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=1,
    weight_decay=0.01,
)

In [15]:
training_arguments

TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_steps=None,
evaluation_strategy=IntervalStrategy.EPOCH,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
hub_model_id=None,
hub_strategy=HubStrategy.EVERY_SAVE,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=2e-05,
length_column_name=length,
load_best_model_at_end=False,
local_rank=-1,
log_level=-1,
log_level_replica=-1,
log_on_each_node=True,
logging_dir=/aiffel/aiffel/transformers/runs/Sep15_02-37-26_wy73nz14myc73iartlx33j2ur-54bdb8549d-sgkzf,
logging_first_s

In [17]:
import datasets

metric = datasets.load_metric('accuracy')

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

In [24]:
trainer = Trainer(
    model=model,
    args=training_arguments,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

In [25]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: id, document.
***** Running training *****
  Num examples = 150000
  Num Epochs = 1
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 37500


Epoch,Training Loss,Validation Loss,Accuracy
1,0.395100,0.389570,0.901040


Saving model checkpoint to /aiffel/aiffel/transformers/checkpoint-500
Configuration saved in /aiffel/aiffel/transformers/checkpoint-500/config.json
Model weights saved in /aiffel/aiffel/transformers/checkpoint-500/pytorch_model.bin
Saving model checkpoint to /aiffel/aiffel/transformers/checkpoint-1000
Configuration saved in /aiffel/aiffel/transformers/checkpoint-1000/config.json
Model weights saved in /aiffel/aiffel/transformers/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to /aiffel/aiffel/transformers/checkpoint-1500
Configuration saved in /aiffel/aiffel/transformers/checkpoint-1500/config.json
Model weights saved in /aiffel/aiffel/transformers/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to /aiffel/aiffel/transformers/checkpoint-2000
Configuration saved in /aiffel/aiffel/transformers/checkpoint-2000/config.json
Model weights saved in /aiffel/aiffel/transformers/checkpoint-2000/pytorch_model.bin
Saving model checkpoint to /aiffel/aiffel/transformers/checkpoi

Saving model checkpoint to /aiffel/aiffel/transformers/checkpoint-18500
Configuration saved in /aiffel/aiffel/transformers/checkpoint-18500/config.json
Model weights saved in /aiffel/aiffel/transformers/checkpoint-18500/pytorch_model.bin
Saving model checkpoint to /aiffel/aiffel/transformers/checkpoint-19000
Configuration saved in /aiffel/aiffel/transformers/checkpoint-19000/config.json
Model weights saved in /aiffel/aiffel/transformers/checkpoint-19000/pytorch_model.bin
Saving model checkpoint to /aiffel/aiffel/transformers/checkpoint-19500
Configuration saved in /aiffel/aiffel/transformers/checkpoint-19500/config.json
Model weights saved in /aiffel/aiffel/transformers/checkpoint-19500/pytorch_model.bin
Saving model checkpoint to /aiffel/aiffel/transformers/checkpoint-20000
Configuration saved in /aiffel/aiffel/transformers/checkpoint-20000/config.json
Model weights saved in /aiffel/aiffel/transformers/checkpoint-20000/pytorch_model.bin
Saving model checkpoint to /aiffel/aiffel/transf

Configuration saved in /aiffel/aiffel/transformers/checkpoint-37000/config.json
Model weights saved in /aiffel/aiffel/transformers/checkpoint-37000/pytorch_model.bin
Saving model checkpoint to /aiffel/aiffel/transformers/checkpoint-37500
Configuration saved in /aiffel/aiffel/transformers/checkpoint-37500/config.json
Model weights saved in /aiffel/aiffel/transformers/checkpoint-37500/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: id, document.
***** Running Evaluation *****
  Num examples = 50000
  Batch size = 4


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=37500, training_loss=0.4177820764160156, metrics={'train_runtime': 17041.4261, 'train_samples_per_second': 8.802, 'train_steps_per_second': 2.201, 'total_flos': 3.9466658304e+16, 'train_loss': 0.4177820764160156, 'epoch': 1.0})

In [28]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: id, document.
***** Running Evaluation *****
  Num examples = 50000
  Batch size = 4


{'eval_loss': 0.3895704746246338,
 'eval_accuracy': 0.90104,
 'eval_runtime': 1686.288,
 'eval_samples_per_second': 29.651,
 'eval_steps_per_second': 7.413,
 'epoch': 1.0}

## Bucketing, Dynamic Padding 추가

In [ ]:
# 전처리시 패딩 미적용
def transform_without_padding(data):
    return tokenizer(
        data['document'],
        truncation=True,
        return_token_type_ids=False,
    )

# 패딩 없는 데이터셋 생성
train_dataset = dataset['train']
test_dataset = dataset['test']

train_dataset = train_dataset.map(transform_without_padding, batched=True)
test_dataset = test_dataset.map(transform_without_padding, batched=True)

In [24]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, pad_to_multiple_of=4)

In [25]:
del model
model = AutoModelForSequenceClassification.from_pretrained("klue/bert-base")

loading configuration file https://huggingface.co/klue/bert-base/resolve/main/config.json from cache at /aiffel/.cache/huggingface/transformers/fbd0b2ef898c4653902683fea8cc0dd99bf43f0e082645b913cda3b92429d1bb.99b3298ed554f2ad731c27cdb11a6215f39b90bc845ff5ce709bb4e74ba45621
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.11.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 32000
}

loading weights file https://huggingface.co/klue/bert-base/resolve/main/pytorch_model.bin from cache at /aiffel/.cache/huggingface/transform

In [26]:
training_arguments = TrainingArguments(
    output_dir,
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    #per_device_train_batch_size=4,
    #per_device_eval_batch_size=4,
    num_train_epochs=1,
    weight_decay=0.01,
    group_by_length=True
)

trainer = Trainer(
    model=model,
    args=training_arguments,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    data_collator=data_collator
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: id, document.
***** Running training *****
  Num examples = 150000
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 18750


Epoch,Training Loss,Validation Loss


Saving model checkpoint to /aiffel/aiffel/transformers/checkpoint-500
Configuration saved in /aiffel/aiffel/transformers/checkpoint-500/config.json
Model weights saved in /aiffel/aiffel/transformers/checkpoint-500/pytorch_model.bin
Saving model checkpoint to /aiffel/aiffel/transformers/checkpoint-1000
Configuration saved in /aiffel/aiffel/transformers/checkpoint-1000/config.json
Model weights saved in /aiffel/aiffel/transformers/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to /aiffel/aiffel/transformers/checkpoint-1500
Configuration saved in /aiffel/aiffel/transformers/checkpoint-1500/config.json
Model weights saved in /aiffel/aiffel/transformers/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to /aiffel/aiffel/transformers/checkpoint-2000
Configuration saved in /aiffel/aiffel/transformers/checkpoint-2000/config.json
Model weights saved in /aiffel/aiffel/transformers/checkpoint-2000/pytorch_model.bin
Saving model checkpoint to /aiffel/aiffel/transformers/checkpoi

In [ ]:
trainer.evaluate()